# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

Answer: This is a classification type problem; we need to classify the students into two groups: those who need intervention and those who don't.

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [3]:
# Import libraries
import numpy as np
import pandas as pd

In [4]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [5]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = len(student_data)
n_features = len(student_data.columns)-1
n_passed = sum(student_data['passed'] == 'yes')
n_failed = sum(student_data['passed'] == 'no')
grad_rate = float(n_passed)*100/n_students
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [6]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [7]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [8]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
train_rows = np.random.choice(X_all.index.values,num_train,replace = False)
X_train = X_all.ix[train_rows]
y_train = y_all.ix[train_rows]

X_test = X_all.drop(train_rows)
y_test = y_all.drop(train_rows)
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [9]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
'''
 We chose 3 potential models: Decision Tree Classifier, SVM Classifier, Gaussian Naive Bayes Classifier. Below are the pros and
 cons of using each model.
| Classifier           | Performance | Speed     | Ease of Use | Ease of Explanation |
|----------------------|-------------|-----------|-------------|---------------------|
| Decision Tree        | Poor        | Fast      | Difficult   | Easy                |
| Linear SVM           | Good        | Slow      | Medium      | Difficult           |
| Gaussian Naive Bayes | Medium      | Medium    | Easy        | Medium              |

Gaussian NB, being a non-parametric model, is the easiest model to use (as it requires no fine tuning) and it has reasonable
performance.

Decision Tree is probably the easiest model to explain. However, it suffers from poor performance and has many
parameters that require fine tuning for an accurate model. It is easy to overfit/underfit decision tree models due to the number
of parameters.

Linear SVMs have the best performance among selected models and have only 1 significant parameter (relative cost of incorrect
classification, 'C') to fine tune, making them easy to use. Unfortunately, SVMs are difficult to explain to a lay person as
they involve transforming the given data before classifying them.

'''
clf_dt = DecisionTreeClassifier()

# Fit model to training data
train_classifier(clf_dt, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

Training DecisionTreeClassifier...
Done!
Training time (secs): 0.003


In [14]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf_dt, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)


Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.931989924433


In [11]:
# Predict on test data
print "F1 score for test set with Decision Tree: {}".format(predict_labels(clf_dt, X_test, y_test))


Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set with Decision Tree: 0.716417910448


In [16]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant
for num_train in [100, 200, 300]:
    train_rows = np.random.choice(X_all.index.values,num_train,replace = False)
    X_train = X_all.ix[train_rows]
    y_train = y_all.ix[train_rows]
    X_test = X_all.drop(train_rows)
    y_test = y_all.drop(train_rows)
    train_predict(clf_dt, X_train, y_train, X_test, y_test)

------------------------------------------
Training set size: 100
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.641909814324
------------------------------------------
Training set size: 200
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.002
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.739837398374
------------------------------------------
Training set size: 300
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.002
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 

In [30]:
clf_svm = LinearSVC()
clf_nb = GaussianNB()
train_classifier(clf_svm, X_train, y_train) # note: using entire training set here with size = 300
train_classifier(clf_nb, X_train, y_train)  # note: using entire training set here with size = 300
train_f1_score = predict_labels(clf_svm, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)
train_f1_score = predict_labels(clf_nb, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)
print "F1 score for test set with SVM: {}".format(predict_labels(clf_svm, X_test, y_test))
print "F1 score for test set with Naive Bayes: {}".format(predict_labels(clf_nb, X_test, y_test))

for num_train in [100, 200, 300]:
    train_rows = np.random.choice(X_all.index.values,num_train,replace = False)
    X_train = X_all.ix[train_rows]
    y_train = y_all.ix[train_rows]
    X_test = X_all.drop(train_rows)
    y_test = y_all.drop(train_rows)
    train_predict(clf_svm, X_train, y_train, X_test, y_test)
    train_predict(clf_nb, X_train, y_train, X_test, y_test)

Training LinearSVC...
Done!
Training time (secs): 0.031
Training GaussianNB...
Done!
Training time (secs): 0.002
Predicting labels using LinearSVC...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.728291316527
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.801909307876
Predicting labels using LinearSVC...
Done!
Prediction time (secs): 0.000
F1 score for test set with SVM: 0.591836734694
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for test set with Naive Bayes: 0.637168141593
------------------------------------------
Training set size: 100
Training LinearSVC...
Done!
Training time (secs): 0.008
Predicting labels using LinearSVC...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.893333333333
Predicting labels using LinearSVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.727272727273
------------------------------------------
Training set siz

| Decision Tree    | Training Time | Prediction Time | F1 - Training | F1 - Test |
|------------------|---------------|-----------------|---------------|-----------|
| Train Size = 100 | 0.001 s       | 0.000 s         | 1.0           | 0.64      |
| Train Size = 200 | 0.002 s       | 0.000 s         | 1             | 0.74      |
| Train Size = 300 | 0.002 s       | 0.001 s         | 1             | 0.72      |

| Gaussian NB      | Training Time | Prediction Time | F1 - Training | F1 - Test |
|------------------|---------------|-----------------|---------------|-----------|
| Train Size = 100 | 0.001 s       | 0.000 s         | 0.59          | 0.39      |
| Train Size = 200 | 0.002 s       | 0.001 s         | 0.79          | 0.77      |
| Train Size = 300 | 0.001 s       | 0.000 s         | 0.81          | 0.77      |

| Linear SVC       | Training Time | Prediction Time | F1 - Training | F1 - Test |
|------------------|---------------|-----------------|---------------|-----------|
| Train Size = 100 | 0.008 s       | 0.001 s         | 0.89          | 0.73      |
| Train Size = 200 | 0.024 s       | 0.001 s         | 0.83          | 0.77      |
| Train Size = 300 | 0.030 s       | 0.000 s         | 0.83          | 0.82      |


## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

At this stage, I'd recommend the Gaussian Naive Bayes model to the board of supervisors.
It has strong performance, doesn't suffer from the risk of being over fitted and can be trained very quickly. It also has the benefit of being easy to explain.

The model makes a prediction for a given student as follows:
- For each possible classification C<sub>i</sub>, the model calculates the probability that the given data belongs to that classification, P<sub>i</sub>. The classification (pass/fail) predicted for a new student is simply one with highest probability.
- To calculate P<sub>i</sub>, the model uses Bayes rule, i.e. the probability of classification given some feature (features are list of data that is known for a student, e.g. 'distance from home', 'Mother's job' etc.) values can be derived from:
    - probability of feature values given the classification; and
    - probability of classification itself
    
Both these probabilities can be derived from the training data. The model makes two simplifying assumptions:
    - For continuous features (non-nominal), the model assumes underlying distribution is Gaussian. 
    - The model assumes all features are independent of each other, i.e. P(X<sub>j</sub>X<sub>k</sub>)= P(X<sub>j</sub>) x
      P(X<sub>k</sub>) for all X<sub>j</sub>, X<sub>k</sub> features in the model.

The second assumption can easily seen to be misleading: e.g. 'health' and 'absences' are likely not independent features. However, despite this limitation, the model performs very well with real life data and is able to predict with high accuracy and precision.

<b> Since Gaussian NB is a non-parametric model, it can't be further fine tuned. To explore fine-tuning, we select a Linear SVC model. </b>


Linear SVC offers the following pros:

1) Reasonable Training and Prediction times: While SVCs are not as fast as Gaussian NBs and Decision Trees, due to small training set, these are not primary concerns.

2) Very strong performance: Linear SVC has comparable performance to Gaussian NB prior to fine tuning. We expect it to outperform the Naive Bayes model post fine tuning.

Prior to making a prediction, the Linear SVC model is trained as follows:
To start, the model assigns a sign +/- to each of the two classes of data (Pass/Fail).

The model uses training data to discover the hyper-plane which optimizes the trade-off between, a) Maximizing the distance between the closest data points belonging to the two classes (pass/fail) and, b) ensuring that all data points are correctly classified. 

Based on the vector w, normal to the hyper-plane, discovered from training data, the prediction of classification of un-seen data is simply sign of (dot product of w and x + b) where x is the unseen data and b is the parameter which determines the offset of the hyper-plane from origin (the offset is b/||w||).

We can fine tune the model by setting the penalty for 'wrong classification' in training data by using parameter 'C'. We do so
below using GridSearchCV to fine tune the model over 10 different values of C.


In [32]:
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import f1_score, make_scorer
def fit_model(clf, X, y):
    ''' Tunes a classification model using GridSearchCV on the input data X 
        and target labels y and returns this optimal model. '''

    
     # Set up the parameters we wish to tune. We will try 10 possible values for parameter C
    parameters = {'C':np.logspace(-20, 10, 10)}

    # Make an appropriate scoring function
    scoring_function = make_scorer(f1_score,greater_is_better = True)

    # Make the GridSearchCV object
    best_clf = GridSearchCV(clf,param_grid = parameters, scoring=scoring_function)

    # Fit the learner to the dataset to obtain the optimal model with tuned parameters
    # Since the fitness requies nominal target classification, we convert 'yes'/'no' to 1/0
    best_clf.fit(X, y== 'yes')

    # Return the optimal model
    return best_clf
#since the last value of training data size was 300, we use that over here. 
best_clf = fit_model(clf_svm, X_train, y_train)
print "Final model optimal parameters:", best_clf.best_params_

Final model optimal parameters: {'C': 9.9999999999999995e-21}


In [33]:
clf_svm = LinearSVC(C = best_clf.best_params_['C'])
train_classifier(clf_svm, X_train, y_train)
test_f1_score = predict_labels(clf_svm, X_test, y_test)
print "Final F1 score for test set using tuned SVC: {}".format(test_f1_score)

Training LinearSVC...
Done!
Training time (secs): 0.001
Predicting labels using LinearSVC...
Done!
Prediction time (secs): 0.001
Final F1 score for test set using tuned SVC: 0.827160493827
